<a href="https://colab.research.google.com/github/aarmintia/imdb/blob/main/Data_Strategist_GCash_part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reference:
https://medium.com/analytics-vidhya/exploratory-data-analysis-imdb-dataset-cff0c3991ad5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_palette("viridis")
import pickle
import glob
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Title Basics Table

In [ ]:
df_title_basics0 = pickle.load(open("/content/drive/MyDrive/extracted/title.basics.sav","rb"))

In [ ]:
df_title_basics_copy = df_title_basics0.copy()

In [ ]:
df_title_basics = df_title_basics_copy[(df_title_basics_copy.titleType == "movie") | (df_title_basics_copy.titleType == "tvMovie")]

In [ ]:
df_title_basics_2020 = df_title_basics[df_title_basics['startYear'] == 2020]
df_title_basics_2020.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
61107,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0.0,2020.0,NaN,70,Drama
220830,tt0230622,movie,Parinati,Parinati,0.0,2020.0,NaN,125,Drama
280921,tt0293513,movie,"Pit Bull: A Tale of Lust, Murder and Revenge",Pit Bull,0.0,2020.0,NaN,NaN,NaN
352668,tt0368133,movie,The Promise of Perfume,The Promise of Perfume,0.0,2020.0,NaN,57,Documentary
420996,tt0438755,movie,About Face: The Story of the Jewish Refugee So...,About Face: The Story of the Jewish Refugee So...,0.0,2020.0,NaN,94,Documentary


In [ ]:
df_title_basics_2020.shape

(18853, 9)

# Ratings Table

In [ ]:
df_ratings = pickle.load(open("/content/drive/MyDrive/extracted/title.ratings.sav","rb"))
df_ratings.head(1)

,tconst,averageRating,numVotes
0,tt0000001,5.7,2033


# Merge 2 tables

In [ ]:
merged_basic_rating = pd.merge(df_title_basics_2020, df_ratings, on='tconst', how='left')

In [ ]:
merged_basic_rating.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0.0,2020.0,NaN,70,Drama,6.4,186.0
1,tt0230622,movie,Parinati,Parinati,0.0,2020.0,NaN,125,Drama,7.8,81.0
2,tt0293513,movie,"Pit Bull: A Tale of Lust, Murder and Revenge",Pit Bull,0.0,2020.0,NaN,NaN,NaN,NaN,NaN
3,tt0368133,movie,The Promise of Perfume,The Promise of Perfume,0.0,2020.0,NaN,57,Documentary,6.6,8.0
4,tt0438755,movie,About Face: The Story of the Jewish Refugee So...,About Face: The Story of the Jewish Refugee So...,0.0,2020.0,NaN,94,Documentary,8.2,45.0


In [ ]:
merged_basic_rating.shape

(18853, 11)

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')

words = merged_basic_rating['originalTitle'].str.lower().str.split().explode()

stop_words = set(stopwords.words('english'))
words = words[~words.isin(stop_words)]

words = words.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

words = words[words.str.len() > 2]

word_counts = words.value_counts().reset_index()
word_counts.columns = ['Word', 'Count']

total_words_count = word_counts['Count'].sum()

word_counts['Percentage'] = (word_counts['Count'] / total_words_count) * 100

word_stats = merged_basic_rating.groupby(words.reset_index(drop=True)).agg(Total_NumVotes=('numVotes', 'sum'), Average_Rating=('averageRating', 'mean')).reset_index()

word_stats = pd.merge(word_counts, word_stats, left_on='Word', right_on='originalTitle', how='left')

word_stats.drop(columns=['originalTitle'], inplace=True)

word_stats_sorted = word_stats.sort_values(by='Count', ascending=False)

print(word_stats_sorted.head(20))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


         Word  Count  Percentage  Total_NumVotes  Average_Rating
0        love    250    0.522783         10690.0        6.124000
1       story    225    0.470505         16990.0        6.522000
2   christmas    202    0.422409         14971.0        6.658333
3         die    188    0.393133         36795.0        6.044444
4         der    186    0.388950         23371.0        6.817857
5        life    165    0.345037         80955.0        6.418182
6       movie    122    0.255118         27464.0        6.147826
7         one    111    0.232116         92441.0        6.025000
8        2020    109    0.227933         20604.0        5.871429
9       world    101    0.211204         19114.0        6.480000
10       last     99    0.207022         17773.0        5.960000
11        les     94    0.196566         56655.0        6.300000
12        man     93    0.194475          5143.0        7.000000
13        del     90    0.188202         18900.0        6.100000
15       home     89    0

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')

words = merged_basic_rating['originalTitle'].str.lower().str.split().explode()

stop_words = set(stopwords.words('english'))
words = words[~words.isin(stop_words)]

words = words.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

words = words[words.str.len() > 2]

word_counts = words.value_counts().reset_index()
word_counts.columns = ['Word', 'Count']

word_stats = merged_basic_rating.groupby(words.reset_index(drop=True)).agg(Total_NumVotes=('numVotes', 'sum'), Average_Rating=('averageRating', 'mean')).reset_index()

word_stats = pd.merge(word_counts, word_stats, left_on='Word', right_on='originalTitle', how='left')

word_stats.drop(columns=['originalTitle'], inplace=True)

total_votes_sum = word_stats['Total_NumVotes'].sum()

word_stats['Total_NumVotes_Percentage'] = (word_stats['Total_NumVotes'] / total_votes_sum) * 100

word_stats_sorted = word_stats.sort_values(by='Total_NumVotes', ascending=False)

print(word_stats_sorted.head(20))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


             Word  Count  Total_NumVotes  Average_Rating  \
186         virus     21        613416.0        5.725000   
106       silence     30        376154.0        6.283333   
2997         biye      3        324209.0        5.300000   
137        united     25        300250.0        6.266667   
21181     absturz      1        269852.0        7.200000   
2256          1st      3        260737.0        6.800000   
2382    portraits      3        252978.0        7.100000   
21437        twos      1        216733.0        6.600000   
21523  britannias      1        209679.0        7.500000   
164          west     23        192919.0        6.366667   
16513       mecüc      1        192448.0        7.700000   
5291          sel      2        189937.0        8.200000   
315        saving     16        183702.0        5.100000   
2901          zoo      3        182188.0        6.700000   
1031        candy      7        181031.0        5.433333   
1492      twisted      5        173919.0

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')

words = merged_basic_rating['originalTitle'].str.lower().str.split().explode()

stop_words = set(stopwords.words('english'))
words = words[~words.isin(stop_words)]

words = words.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

words = words[words.str.len() > 2]

word_counts = words.value_counts().reset_index()
word_counts.columns = ['Word', 'Count']

word_stats = merged_basic_rating.groupby(words.reset_index(drop=True)).agg(Total_NumVotes=('numVotes', 'sum'), Average_Rating=('averageRating', 'mean')).reset_index()

word_stats = pd.merge(word_counts, word_stats, left_on='Word', right_on='originalTitle', how='left')

word_stats.drop(columns=['originalTitle'], inplace=True)

total_votes_sum = word_stats['Average_Rating'].sum()

word_stats['Average_Rating_Percentage'] = (word_stats['Average_Rating'] / total_votes_sum) * 100

word_stats = word_stats[word_stats['Total_NumVotes'] > 250000]

word_stats_sorted = word_stats.sort_values(by='Average_Rating', ascending=False)

print(word_stats_sorted.head(20))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


            Word  Count  Total_NumVotes  Average_Rating  \
21181    absturz      1        269852.0        7.200000   
2382   portraits      3        252978.0        7.100000   
2256         1st      3        260737.0        6.800000   
106      silence     30        376154.0        6.283333   
137       united     25        300250.0        6.266667   
186        virus     21        613416.0        5.725000   
2997        biye      3        324209.0        5.300000   

       Average_Rating_Percentage  
21181                   0.018274  
2382                    0.018020  
2256                    0.017259  
106                     0.015948  
137                     0.015905  
186                     0.014531  
2997                    0.013452  
